In [1]:
# import modules
import pybamm
import numpy as np;import matplotlib.pyplot as plt;
from pybamm import constants,exp,tanh,sqrt;

from pybamm import exp,sqrt
def electrolyte_diffusivity_Nyman2008Constant(c_e, c_EC,T):
    D_c_e = (
        (c_e <= 2000) * (
            8.794e-11 * (c_e / 1000) ** 2 - 3.972e-10 * (c_e / 1000) + 4.862e-10) 
        + (c_e > 2000) * (
        8.794e-11 * (2000 / 1000) ** 2 - 3.972e-10 * (2000 / 1000) + 4.862e-10)
    )
    return D_c_e

def electrolyte_conductivity_Nyman2008Constant(c_e,c_EC, T):
    sigma_e = (c_e <= 2000) * (
        0.1297 * (c_e / 1000) ** 3 - 2.51 * (c_e / 1000) ** 1.5 + 3.329 * (c_e / 1000)
    ) + (c_e > 2000) *  (
        0.1297 * (2000 / 1000) ** 3 - 2.51 * (2000 / 1000) ** 1.5 + 3.329 * (2000 / 1000)
    )
    return sigma_e
print(electrolyte_diffusivity_Nyman2008Constant(10,4541,298.15))
print(electrolyte_conductivity_Nyman2008Constant(10,4541,298.15))


4.82236794e-10
0.0307801297


In [2]:

# set-up and run model
V_max = 4.2;        V_min = 2.5;
total_cycle = 1; save_at_cycles= 1
Experiment_Long   = pybamm.Experiment( [ (
    f"Discharge at 1 C until {V_min} V (1 minutes period)" , )] * total_cycle ) 
Experiment_s   = pybamm.Experiment( [ (
    f"Discharge at 1 C for 20 minutes (10 seconds period)" , )] * total_cycle ) 

Sim = [];Sol=[]; 

ChemistryChen=pybamm.parameter_sets.Chen2020 
ChemistryChen["electrolyte"] = "lipf6_Nyman2008_rio";
Para_0=pybamm.ParameterValues(chemistry=ChemistryChen);

Para_0['Negative electrode porosity'] =  0.1
var = pybamm.standard_spatial_vars  
mesh_thick = 30; mesh_par=30;
var_pts = {
    var.x_n: int(mesh_thick),  
    var.x_s: int(20),  
    var.x_p: int(mesh_thick),  
    var.r_n: int(mesh_par),  
    var.r_p: int(mesh_par),  }  

pybamm.settings.tolerances['D_e__c_e'] = 10


In [3]:

# Run the first model, with Nyman2008 polynimial form
model = pybamm.lithium_ion.DFN()
c_e = model.variables["Electrolyte concentration [mol.m-3]"]
c_EC = model.variables["EC concentration [mol.m-3]"]
T = model.variables["Cell temperature [K]"]
model.variables["Electrolyte conductivity [S.m-1]"] =(
    Para_0['Electrolyte conductivity [S.m-1]'](c_e, c_EC,T))
model.variables["Electrolyte diffusivity [m2.s-1]"] =(
    Para_0['Electrolyte diffusivity [m2.s-1]'](c_e, c_EC,T))

sim = pybamm.Simulation(
    model, experiment = Experiment_Long,
    parameter_values = Para_0,
    solver = pybamm.CasadiSolver(),
    var_pts=var_pts,)  
try:
    sol = sim.solve(save_at_cycles=save_at_cycles,);
    print(sol.cycles[-1].steps[-1]);  # a way to check whether the solution is finalized 
except:
    print('Fail for electrolyte: Poly')
else:
    Sol.append(sol)   
    print('Succeed for electrolyte: Poly')


Succeed for electrolyte: Poly


In [4]:
# Run the first model, with Nyman2008 exponeitial form
model_2 = pybamm.lithium_ion.DFN()
Para_0.update({
    'Electrolyte conductivity [S.m-1]': 
    electrolyte_conductivity_Nyman2008Constant})
Para_0.update({
    'Electrolyte diffusivity [m2.s-1]': 
    electrolyte_diffusivity_Nyman2008Constant})
c_e = model_2.variables["Electrolyte concentration [mol.m-3]"]
c_EC = model.variables["EC concentration [mol.m-3]"]
T = model_2.variables["Cell temperature [K]"]
model_2.variables["Electrolyte conductivity [S.m-1]"] =(
    Para_0['Electrolyte conductivity [S.m-1]'](c_e,c_EC, T))
model_2.variables["Electrolyte diffusivity [m2.s-1]"] =(
    Para_0['Electrolyte diffusivity [m2.s-1]'](c_e,c_EC, T))

sim_2 = pybamm.Simulation(
    model_2, experiment = Experiment_Long,
    parameter_values = Para_0,
    solver = pybamm.CasadiSolver(),
    var_pts=var_pts,)  
try:
    sol_2 = sim_2.solve(save_at_cycles=save_at_cycles,);
    print(sol_2.cycles[-1].steps[-1]);  # a way to check whether the solution is finalized 
except:
    print('Fail for electrolyte: Constant')
else:
    Sol.append(sol_2)   
    print('Succeed for electrolyte: Constant')


Succeed for electrolyte: Constant


In [10]:
print(Sol[0]["Electrolyte conductivity [S.m-1]"].entries[:,-1])
print(Sol[0]["Electrolyte concentration [mol.m-3]"].entries[:,-1])

[2.3210205  2.31228375 2.29481611 2.26862951 2.2337424  2.19018033
 2.13797691 2.07717517 2.00782939 1.93000786 1.8437967  1.74930554
 1.64667596 1.5360946  1.41781399 1.29218752 1.15973072 1.02123691
 0.87801457 0.73243763 0.5894674  0.46230464 0.40901407 0.58513846
 0.81236412 0.91151796 0.94712983 0.94356241 0.91096714 0.85442148
 0.81592336 0.81429867 0.81266613 0.81102576 0.80937759 0.80772163
 0.80605791 0.80438645 0.80270726 0.80102037 0.7993258  0.79762357
 0.79591369 0.79419618 0.79247107 0.79073838 0.78899811 0.78725029
 0.78549494 0.78373208 0.77664694 0.76434887 0.752149   0.74008507
 0.72819259 0.71650499 0.70505381 0.69386877 0.68297795 0.67240784
 0.66218346 0.65232839 0.64286487 0.63381382 0.6251949  0.61702653
 0.60932591 0.60210907 0.59539085 0.58918494 0.58350385 0.57835896
 0.57376046 0.56971742 0.5662377  0.56332805 0.56099398 0.55923987
 0.55806888 0.557483  ]
[4327.31441956 4324.17086076 4317.85945041 4308.33081317 4295.508807
 4279.28783617 4259.52894308 4236.05

In [9]:
print(Sol[-1]["Electrolyte conductivity [S.m-1]"].entries[:,-1])
print(Sol[-1]["Electrolyte concentration [mol.m-3]"].entries[:,-1])

[0.59624792 0.59624792 0.59624792 0.59624792 0.59624792 0.59624792
 0.59624792 0.59624792 0.59624792 0.59624792 0.59624792 0.59624792
 0.59624792 0.59624792 0.59624792 0.59624792 0.59624792 0.59624792
 0.59624792 0.78021067 0.89989171 0.94426257 0.94654367 0.91969659
 0.87014507 0.80142739 0.71550696 0.61329986 0.49483327 0.3590902
 0.28094701 0.27771077 0.27446565 0.27121159 0.26794851 0.26467638
 0.26139512 0.25810467 0.25480496 0.25149594 0.24817752 0.24484964
 0.24151224 0.23816522 0.23480852 0.23144206 0.22806575 0.22467951
 0.22128326 0.21787691 0.20420804 0.18084448 0.15804267 0.13589104
 0.11449207 0.09396773 0.07446855 0.05618973 0.03940509 0.02454385
 0.01240545 0.00373877        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]
[ 9.45704628e+03  9.40956928e+03  9.31489640e+03  9.17354225e+03
  8.98616400e+03  8.75344202e+03  8.47595

In [5]:

label = ['Nyman - Poly', 'Nyman - Constant', ]
output_variables3 = [
    "Terminal voltage [V]",   
    "Electrolyte potential [V]",
    "Electrolyte concentration [mol.m-3]",
    "Li+ flux [mol.m-2.s-1]",
    "Electrolyte diffusivity [m2.s-1]",
    "Electrolyte conductivity [S.m-1]"
]
quick_plot = pybamm.QuickPlot(
    [sol for sol in Sol], 
    output_variables3,label,
    variable_limits='tight',time_unit='minutes',n_rows=2,
    figsize = (12,8)) #     spatial_unit='mm',
quick_plot.dynamic_plot();


interactive(children=(FloatSlider(value=0.0, description='t', max=58.89902678996891, step=0.5889902678996891),…